In [1]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableParallel

In [2]:
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace

llm = HuggingFaceEndpoint(
    repo_id="meta-llama/Meta-Llama-3-8B-Instruct",
    task="text-generation",
    max_new_tokens=100,
    do_sample=False,
    repetition_penalty=1.03,
)

chat = ChatHuggingFace(llm=llm, verbose=True)

In [3]:
chat_template_books = ChatPromptTemplate.from_template(
    '''
    Suggest three of the best intermediate-level {programming language} books. 
    Answer only by listing the books.
    '''
)

chat_template_projects = ChatPromptTemplate.from_template(
    '''
    Suggest three interesting {programming language} projects suitable for intermediate-level programmers. 
    Answer only by listing the projects.
    '''
)

In [4]:
string_parser = StrOutputParser()

In [5]:
chain_books = chat_template_books | chat | string_parser

chain_projects = chat_template_projects | chat | string_parser

In [6]:
chain_parallel = RunnableParallel({'books':chain_books, 'projects':chain_projects})


In [7]:
chain_parallel.invoke({'programming language':'Python'})

{'books': 'Here are three intermediate-level Python books:\n\n1. "Automate the Boring Stuff with Python" by Al Sweigart\n2. "Python Crash Course" by Eric Matthes\n3. "Python for Data Analysis" by Wes McKinney',
 'projects': 'Here are three interesting Python projects suitable for intermediate-level programmers:\n\n1. Building a Chatbot using Natural Language Processing (NLP)\n2. Creating a Web Scraper to Extract Data from a Website\n3. Developing a Game using Pygame or a similar Game Development Library'}

In [8]:
chain_parallel.get_graph().print_ascii()

               +-------------------------------+                 
               | Parallel<books,projects>Input |                 
               +-------------------------------+                 
                    ***                  ***                     
                ****                        ****                 
              **                                **               
+--------------------+                   +--------------------+  
| ChatPromptTemplate |                   | ChatPromptTemplate |  
+--------------------+                   +--------------------+  
           *                                        *            
           *                                        *            
           *                                        *            
  +-----------------+                     +-----------------+    
  | ChatHuggingFace |                     | ChatHuggingFace |    
  +-----------------+                     +-----------------+    
          

In [9]:
%%time
chain_books.invoke({'programming language':'Python'})

CPU times: total: 15.6 ms
Wall time: 669 ms


'Here are three intermediate-level Python books:\n\n1. "Automate the Boring Stuff with Python" by Al Sweigart\n2. "Python Crash Course" by Eric Matthes\n3. "Python for Data Analysis" by Wes McKinney'

In [10]:
%%time
chain_projects.invoke({'programming language':'Python'})

CPU times: total: 31.2 ms
Wall time: 222 ms


'Here are three interesting Python projects suitable for intermediate-level programmers:\n\n1. Building a Chatbot using Natural Language Processing (NLP)\n2. Creating a Web Scraper to Extract Data from a Website\n3. Developing a Game using Pygame or a similar Game Development Library'

In [11]:
%%time
chain_parallel.invoke({'programming language':'Python'})

CPU times: total: 15.6 ms
Wall time: 760 ms


{'books': 'Here are three intermediate-level Python books:\n\n1. "Automate the Boring Stuff with Python" by Al Sweigart\n2. "Python Crash Course" by Eric Matthes\n3. "Python for Data Analysis" by Wes McKinney',
 'projects': 'Here are three interesting Python projects suitable for intermediate-level programmers:\n\n1. Building a Chatbot using Natural Language Processing (NLP)\n2. Creating a Web Scraper to Extract Data from a Website\n3. Developing a Game using Pygame or a similar Game Development Library'}

# Piping a RunnableParallel with other Runnables

In [12]:
chat_template_time = ChatPromptTemplate.from_template(
     '''
     I'm an intermediate level programmer.
     
     Consider the following literature:
     {books}
     
     Also, consider the following projects:
     {projects}
     
     Roughly how much time would it take me to complete the literature and the projects?
     
     '''
)

In [13]:
chain_time2 = ({'books':chain_books, 
                'projects':chain_projects}
              | chat_template_time 
              | chat 
              | string_parser
             )

by default it forms this object                    
chain_parallel = RunnableParallel({'books':chain_books, 'projects':chain_projects})


In [15]:
print(chain_time2.invoke({'programming language':'Python'}))

What a great combination of literature and projects!

To estimate the time it takes to complete the literature and projects, I'll break them down into smaller components and provide a rough estimate of the time required for each.

**Literature:**

1. "Automate the Boring Stuff with Python" by Al Sweigart: 2-3 weeks ( approx. 40-60 hours)
This book focuses on practical applications of Python programming, so you'll need to spend time practicing and working on


In [16]:
chain_time2.get_graph().print_ascii()

               +-------------------------------+                 
               | Parallel<books,projects>Input |                 
               +-------------------------------+                 
                    ***                  ***                     
                ****                        ****                 
              **                                **               
+--------------------+                   +--------------------+  
| ChatPromptTemplate |                   | ChatPromptTemplate |  
+--------------------+                   +--------------------+  
           *                                        *            
           *                                        *            
           *                                        *            
  +-----------------+                     +-----------------+    
  | ChatHuggingFace |                     | ChatHuggingFace |    
  +-----------------+                     +-----------------+    
          

# RunnableLambda


In [18]:
find_sum = lambda x: sum(x)
find_sum([1, 2, 5])

8

In [19]:
find_square = lambda x: x**2
find_square(8)

64

In [21]:
from langchain_core.runnables import RunnableLambda

In [22]:
runnable_sum = RunnableLambda(lambda x: sum(x))
runnable_sum.invoke([1, 2, 5])

8

In [23]:
runnable_square = RunnableLambda(lambda x: x**2)
runnable_square.invoke(8)

64

In [24]:
chain = runnable_sum | runnable_square

In [25]:
chain.invoke([1, 2, 5])

64

In [26]:
chain.get_graph().print_ascii()

+-------------+  
| LambdaInput |  
+-------------+  
        *        
        *        
        *        
   +--------+    
   | Lambda |    
   +--------+    
        *        
        *        
        *        
   +--------+    
   | Lambda |    
   +--------+    
        *        
        *        
        *        
+--------------+ 
| LambdaOutput | 
+--------------+ 


In [27]:
chain.get_graph().print_ascii()

+-------------+  
| LambdaInput |  
+-------------+  
        *        
        *        
        *        
   +--------+    
   | Lambda |    
   +--------+    
        *        
        *        
        *        
   +--------+    
   | Lambda |    
   +--------+    
        *        
        *        
        *        
+--------------+ 
| LambdaOutput | 
+--------------+ 
